In [2]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime, date
import time



In [17]:

url = 'https://fantasy.premierleague.com/api/entry/956051/history/'
r = requests.get(url).json()
avg:int = (r['past'][-1]['rank'] + r['past'][-2]['rank'] + r['past'][-3]['rank']) / 3
avg

624609.0

In [214]:
def getPlayerInfo(playerId): 
    url2 = 'https://fantasy.premierleague.com/api/event/' + str(thisGw) + '/live/'
    liveInfo = requests.get(url2).json()['elements']
    liveInfo = liveInfo[playerId-1]['explain']
    playerInfo = []
    for stats in liveInfo:
        for stat in stats['stats']:
            playerInfo.append(stat)

    df = pd.DataFrame(playerInfo)
    test = []
    visited = []
    for i in range(len(df)):
        tempIdentifier = df.at[i, 'identifier']
        if tempIdentifier not in visited:
            tempValue = df.loc[df['identifier'] == tempIdentifier, 'value'].sum()
            test.append({
                'identifier' : tempIdentifier,
                'value' : tempValue
            })
            visited.append(tempIdentifier)
    return test


In [216]:
getPlayerInfo(1)

[{'identifier': 'minutes', 'value': 0}]

In [190]:
import collections, functools, operator

In [192]:
playerInfo

[{'identifier': 'minutes', 'points': 0, 'value': 0},
 {'identifier': 'minutes', 'points': 2, 'value': 90},
 {'identifier': 'goals_scored', 'points': 8, 'value': 2},
 {'identifier': 'bonus', 'points': 2, 'value': 2}]

In [207]:
playerInfo[0].values()

dict_values(['minutes', 0, 0])

In [135]:
df = pd.DataFrame(playerInfo)
df

,identifier,points,value
0,minutes,0,0
1,minutes,2,90
2,goals_scored,8,2
3,bonus,2,2


In [211]:
test = []
visited = []
for i in range(len(df)):
    tempIdentifier = df.at[i, 'identifier']
    if tempIdentifier not in visited:
        tempValue = df.loc[df['identifier'] == tempIdentifier, 'value'].sum()
        test.append({
            'identifier' : tempIdentifier,
            'value' : tempValue
        })
        visited.append(tempIdentifier)
test

[{'identifier': 'minutes', 'value': 90},
 {'identifier': 'goals_scored', 'value': 2},
 {'identifier': 'bonus', 'value': 2}]

In [134]:
for stats in liveInfo:
    for stat in stats['stats']:
        tempList = list(stat.keys())
         

In [182]:
test[0]['identifier']

'goals_scored'